In [121]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [159]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv('/content/drive/MyDrive/movies.csv', sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [123]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [124]:
ratings.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


### Content Based

**Objective**: Build a Content-Based Recommendation system that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

**Dataset**:
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Split the `genres` column into separate genre strings.
   - Fill any missing values in the `genres` column and convert it to string.

2. **Compute TF-IDF Matrix**:
   - Use `TfidfVectorizer` to transform the genres into a TF-IDF matrix.

3. **Calculate Cosine Similarity**:
   - Compute cosine similarity between the TF-IDF matrices of movies.

4. **Recommend Movies**:
   - Create a function that gets movie recommendations based on the cosine similarity score of movie genres.

In [125]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [127]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [128]:
indices = pd.Series(movies.index, index=movies['title']).drop_duplicates()

In [129]:
def genre_recommendations(title, n=10):
    sim_scores = sorted(list(enumerate(cosine_sim[indices[title]])), key=lambda x: x[1], reverse=True)[1:n+1]
    movie_indices = [i[0] for i in sim_scores]
    return movies['title'].iloc[movie_indices]

In [130]:
# Example
genre_recommendations('GoldenEye (1995)')

345     Clear and Present Danger (1994)
543           Surviving the Game (1994)
724                    Rock, The (1996)
788                     Daylight (1996)
825               Chain Reaction (1996)
978                 Maximum Risk (1996)
1467                    Anaconda (1997)
1513                     Con Air (1997)
1693                   Firestorm (1998)
3686          Perfect Storm, The (2000)
Name: title, dtype: object


### Collaborative Filtering

**Objective**: Implement a collaborative filtering movie recommendation system to recommend movies to a user based on similar users' spreferences.

**Dataset**:

- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Split `genres` column into separate rows.

2. **Create User-Genre Matrix**:
   - Create a matrix where rows are users and columns are genres.
   - Each cell represents the ratio of movies watched by the user in that genre to the total movies watched by the user.

3. **Calculate User Similarity**:
   - Use cosine similarity to calculate the similarity between users.

4. **Find Top N Similar Users**:
   - Find the top N users with the most similar preferences to the given user.

5. **Recommend Movies**:
   - Recommend movies that similar users have watched but the target user has not.
   - Return titles and genres of the top 10 recommended movies.

In [131]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [161]:
data = pd.merge(ratings, movies, on='movie_id')

In [162]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),Drama


In [163]:
data = data.assign(genres=data['genres'].str.split('|')).explode('genres')

In [164]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),Drama


In [165]:
user_genre_matrix = data.pivot_table(index='user_id', columns='genres', aggfunc='size', fill_value=0)
user_genre_matrix = user_genre_matrix.div(user_genre_matrix.sum(axis=1), axis=0)

user_similarity = cosine_similarity(user_genre_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=user_genre_matrix.index, columns=user_genre_matrix.index)

In [166]:
def get_top_n_similar_users(user_id, n=5):
    sim_scores = user_similarity_df[user_id].sort_values(ascending=False)
    top_n_users = sim_scores.index[1:n+1]
    return top_n_users

In [167]:
def recommend_movies(user_id, n_similar_users=5, n_recommendations=10):
    similar_users = get_top_n_similar_users(user_id, n_similar_users)
    similar_users_movies = data[data['user_id'].isin(similar_users)]

    target_user_movies = data[data['user_id'] == user_id]['movie_id'].unique()
    recommendations = similar_users_movies[~similar_users_movies['movie_id'].isin(target_user_movies)]
    top_recommendations = recommendations['movie_id'].value_counts().head(n_recommendations).index
    recommended_movies = movies[movies['movie_id'].isin(top_recommendations)]

    return recommended_movies[['title', 'genres']]

In [168]:
# Example
user_id = 1
recommended_movies = recommend_movies(user_id, n_similar_users=5, n_recommendations=10)

In [169]:
recommended_movies

,title,genres
33,Babe (1995),Children's|Comedy|Drama
900,Casablanca (1942),Drama|Romance|War
1019,Alice in Wonderland (1951),Animation|Children's|Musical
1262,Fantasia (1940),Animation|Children's|Musical
1575,L.A. Confidential (1997),Crime|Film-Noir|Mystery|Thriller
2009,"Jungle Book, The (1967)",Animation|Children's|Comedy|Musical
2011,Lady and the Tramp (1955),Animation|Children's|Comedy|Musical|Romance
2012,"Little Mermaid, The (1989)",Animation|Children's|Comedy|Musical|Romance
2018,Peter Pan (1953),Animation|Children's|Fantasy|Musical
2027,Sleeping Beauty (1959),Animation|Children's|Musical


### SVD (Singular Value Decomposition)



**Objective**: Implement an SVD-based recommendation system to recommend movies to users by decomposing the user-item interaction matrix into latent factors.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Create a user-item matrix where rows represent users, columns represent movies, and the values are the ratings.

2. **Decompose Matrix using SVD**:
   - Apply Singular Value Decomposition (SVD) to decompose the user-item matrix into three matrices: $U$, $\Sigma$, and $V^T$.

3. **Reconstruct Matrix**:
   - Reconstruct the user-item matrix using the top $k$ singular values to reduce dimensionality.

4. **Predict Ratings**:
   - Use the reconstructed matrix to predict ratings for all user-item pairs.

5. **Recommend Movies**:
   - Recommend the top 20 movies with the highest predicted ratings for a given user that the user hasn't rated yet.

In [141]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


Fill na elements.

In [142]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
# calculate rating matrix
from scipy.sparse.linalg import svds

In [144]:
U, sigma, Vt = svds(np.array(Ratings), k=600)
sigma = np.diag(sigma)

In [145]:
all_user_predicted_ratings = U @ sigma @ Vt

In [146]:
preds = pd.DataFrame(all_user_predicted_ratings, columns=Ratings.columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.759176,0.160630,0.102102,0.205702,-0.030151,-0.018541,0.233804,-0.117964,0.017798,-0.048771,...,0.063206,-0.044127,0.043386,-0.091835,0.009510,0.103084,-0.220223,-0.022412,0.056879,0.007557
1,0.099960,-0.342998,0.522385,0.006534,0.319766,-0.010940,0.185944,0.014379,0.084776,0.584475,...,-0.036830,0.004546,-0.038496,0.051936,0.047879,-0.193221,-0.437215,-0.080270,-0.072302,0.099547
2,0.251561,0.262651,0.016005,-0.038401,0.053764,-0.117261,0.271081,-0.065385,0.064760,0.615519,...,0.101062,0.012767,0.044776,-0.036511,0.033302,-0.072931,0.226350,0.046456,-0.041738,-0.178138
3,-0.064985,0.216236,0.044158,-0.041027,0.151929,-0.038368,0.027633,0.000751,-0.005266,-0.014834,...,-0.072918,0.042006,0.024820,-0.062895,-0.023484,-0.030037,-0.001735,0.056297,-0.024846,-0.237383
4,-0.419595,-0.521540,-0.209963,0.073975,-0.079807,1.555207,0.328889,0.087710,-0.142887,0.236268,...,0.093257,0.044767,-0.117749,-0.069421,-0.135470,0.291728,0.376418,0.106066,-0.206397,-0.414821


In [147]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False)

    user_data = original_ratings[original_ratings.user_id == userID]
    user_full = user_data.merge(movies, how='left', on='movie_id').sort_values(['rating'], ascending=False)

    print(f'User {userID} has already rated {user_full.shape[0]} movies.')
    print(f'Recommending highest {num_recommendations} predicted ratings movies not already rated.')

    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
                       merge(pd.DataFrame(sorted_user_predictions).reset_index(), how='left',
                             left_on='movie_id',
                             right_on='movie_id').
                       rename(columns={user_row_number: 'Predictions'}).
                       sort_values('Predictions', ascending=False).
                       iloc[:num_recommendations, :-1])

    return user_full, recommendations

In [148]:
already_rated, predictions = recommend_movies(preds, 4375, movies, ratings, 20)

User 4375 has already rated 325 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [149]:
ratings.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354
5,1,1197,3,0,1196
6,1,1287,5,0,1286
7,1,2804,5,0,2803
8,1,594,4,0,593
9,1,919,4,0,918


In [150]:
# Top 20 movies that User 4375 has rated
already_rated.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
195,4375,3250,5,4374,3249,Alive (1993),['Drama']
285,4375,175,5,4374,174,Kids (1995),['Drama']
199,4375,3257,5,4374,3256,"Bodyguard, The (1992)","['Action', 'Drama', 'Romance', 'Thriller']"
65,4375,3809,5,4374,3808,What About Bob? (1991),['Comedy']
210,4375,2688,5,4374,2687,"General's Daughter, The (1999)","['Drama', 'Thriller']"
270,4375,150,5,4374,149,Apollo 13 (1995),['Drama']
271,4375,2710,5,4374,2709,"Blair Witch Project, The (1999)",['Horror']
125,4375,349,5,4374,348,Clear and Present Danger (1994),"['Action', 'Adventure', 'Thriller']"
60,4375,288,5,4374,287,Natural Born Killers (1994),"['Action', 'Thriller']"
141,4375,47,5,4374,46,Seven (Se7en) (1995),"['Crime', 'Thriller']"


#### Evaluation

In [151]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

train_ratings = train_data.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)
R_train = train_ratings.values
user_ratings_mean_train = np.mean(R_train, axis=1)
Ratings_demeaned_train = R_train - user_ratings_mean_train.reshape(-1, 1)

U_train, sigma_train, Vt_train = svds(Ratings_demeaned_train, k=600)
sigma_train = np.diag(sigma_train)

all_user_predicted_ratings_train = np.dot(np.dot(U_train, sigma_train), Vt_train) + user_ratings_mean_train.reshape(-1, 1)
preds_train = pd.DataFrame(all_user_predicted_ratings_train, columns=train_ratings.columns)

def predict_rating(user_id, movie_id):
    try:
        user_row_number = user_id - 1  # User ID starts at 1, not 0
        return preds_train.loc[user_row_number, movie_id]
    except KeyError:
        return np.nan

test_data['predicted_rating'] = test_data.apply(lambda x: predict_rating(x['user_id'], x['movie_id']), axis=1)

test_data.dropna(inplace=True)

rmse = sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 3.5470134518925924


### Nueral Network Model (Recommender Model)



**Objective**: Implement a Recommender model to recommend movies to a user based on similar users' preferences.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Define Dataset and DataLoader**:
   - Create a custom PyTorch `Dataset` for ratings.
   - Create a DataLoader for batching and shuffling data.

2. **Define the Neural Network**:
   - Create a neural network with embedding layers for users and movies.

3. **Train the Model**:
   - Train the model using Mean Squared Error loss and Adam optimizer.
   - Save model checkpoints.

4. **Evaluate the Model**:
   - Calculate RMSE on the entire dataset.

5. **Predict Ratings for Unrated Movies**:
   - Predict and recommend top 10 unrated movies for a given user.

In [152]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
import torch.optim as optim

In [153]:
# Define the dataset
class RatingsDataset(Dataset):
    def __init__(self, ratings):
        self.user_ids = ratings['user_id'].values
        self.movie_ids = ratings['movie_id'].values
        self.ratings = ratings['rating'].values

    def __len__(self):
        return len(self.user_ids)

    def __getitem__(self, idx):
        return self.user_ids[idx], self.movie_ids[idx], self.ratings[idx]

# Define the neural network
class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=50):
        super(RecommenderNet, self).__init__()
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        self.fc = nn.Linear(embedding_size * 2, 1)

    def forward(self, user, movie):
        user_embedded = self.user_embedding(user)
        movie_embedded = self.movie_embedding(movie)
        concatenated = torch.cat((user_embedded, movie_embedded), dim=1)
        output = self.fc(concatenated)
        return output.squeeze()


# Create the dataset and dataloader
ratings_dataset = RatingsDataset(ratings)
dataloader = DataLoader(ratings_dataset, batch_size=64, shuffle=True)

# Initialize the model, loss function, and optimizer
num_users = ratings['user_id'].nunique()
num_movies = ratings['movie_id'].nunique()
model = RecommenderNet(num_users+500, num_movies+500)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    i = 0
    for user, movie, rating in dataloader:
        optimizer.zero_grad()
        output = model(user, movie)
        loss = criterion(output, rating.float())
        loss.backward()
        optimizer.step()
        i += 1
        if i == 1000:
          break
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 2.2662558555603027
Epoch 2, Loss: 1.0816864967346191
Epoch 3, Loss: 1.3488779067993164
Epoch 4, Loss: 1.1534782648086548
Epoch 5, Loss: 1.0712363719940186
Epoch 6, Loss: 1.0859270095825195
Epoch 7, Loss: 0.9372642636299133
Epoch 8, Loss: 0.9438140392303467
Epoch 9, Loss: 0.6767373085021973
Epoch 10, Loss: 0.8823217153549194


In [154]:
# Evaluation
def evaluate_model(model, dataloader):
    model.eval()
    predictions, targets = [], []

    with torch.no_grad():
        for user, movie, rating in dataloader:
            output = model(user, movie)
            predictions.extend(output.tolist())
            targets.extend(rating.tolist())

    rmse = sqrt(mean_squared_error(targets, predictions))
    return rmse

train_rmse = evaluate_model(model, dataloader)
print(f'Train RMSE: {train_rmse}')


Train RMSE: 0.9256515000993703


In [155]:
# Example

sample_user_id = 1
sample_user_data = ratings[ratings['user_id'] == sample_user_id]
sample_dataset = RatingsDataset(sample_user_data)
sample_dataloader = DataLoader(sample_dataset, batch_size=1, shuffle=False)

print(f'Sample predictions for user ID {sample_user_id}:')
with torch.no_grad():
    for user, movie, rating in sample_dataloader:
        output = model(user, movie).squeeze()
        print(f'Movie ID: {movie.item()}, Predicted Rating: {int(output.item())}, Actual Rating: {rating.item()}')

Sample predictions for user ID 1:
Movie ID: 1193, Predicted Rating: 4, Actual Rating: 5
Movie ID: 661, Predicted Rating: 3, Actual Rating: 3
Movie ID: 914, Predicted Rating: 4, Actual Rating: 3
Movie ID: 3408, Predicted Rating: 4, Actual Rating: 4
Movie ID: 2355, Predicted Rating: 3, Actual Rating: 5
Movie ID: 1197, Predicted Rating: 4, Actual Rating: 3
Movie ID: 1287, Predicted Rating: 4, Actual Rating: 5
Movie ID: 2804, Predicted Rating: 4, Actual Rating: 5
Movie ID: 594, Predicted Rating: 3, Actual Rating: 4
Movie ID: 919, Predicted Rating: 4, Actual Rating: 4
Movie ID: 595, Predicted Rating: 3, Actual Rating: 5
Movie ID: 938, Predicted Rating: 3, Actual Rating: 4
Movie ID: 2398, Predicted Rating: 4, Actual Rating: 4
Movie ID: 2918, Predicted Rating: 4, Actual Rating: 4
Movie ID: 1035, Predicted Rating: 3, Actual Rating: 5
Movie ID: 2791, Predicted Rating: 3, Actual Rating: 4
Movie ID: 2687, Predicted Rating: 3, Actual Rating: 3
Movie ID: 2018, Predicted Rating: 3, Actual Rating: 4


In [156]:
def predict_unrated_movies(user_id, model, ratings, movies):
    user_rated_movies = ratings[ratings['user_id'] == user_id]['movie_id'].values
    unrated_movies = movies[~movies['movie_id'].isin(user_rated_movies)]
    unrated_movie_ids = unrated_movies['movie_id'].values
    user_ids = torch.tensor([user_id] * len(unrated_movie_ids), dtype=torch.long)
    movie_ids = torch.tensor(unrated_movie_ids, dtype=torch.long)

    with torch.no_grad():
        ratings = model(user_ids, movie_ids)

    unrated_movies['predicted_rating'] = ratings.numpy()
    recommended_movies = unrated_movies.sort_values(by='predicted_rating', ascending=False).head(10)
    return recommended_movies

# Example
user_id = 1
predictions = predict_unrated_movies(user_id, model, ratings, movies)
print(predictions)

      movie_id                                              title  \
125        127  Silence of the Palace, The (Saimt el Qusur) (1...   
3312      3381                   Slaves to the Underground (1997)   
315        318                   Shawshank Redemption, The (1994)   
892        904                                 Rear Window (1954)   
2540      2609              King of Masks, The (Bian Lian) (1996)   
847        858                              Godfather, The (1972)   
1186      1204                          Lawrence of Arabia (1962)   
1200      1218          Killer, The (Die xue shuang xiong) (1989)   
846        857                          Crows and Sparrows (1949)   
556        560                  Beans of Egypt, Maine, The (1994)   

                            genres  predicted_rating  
125                      ['Drama']          4.904079  
3312           ['Comedy', 'Drama']          4.708793  
315                      ['Drama']          4.668880  
892        ['Mystery

<ipython-input-156-d195b6894fea>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unrated_movies['predicted_rating'] = ratings.numpy()


### GMM (Gaussian Mixture Model)

**Objective**: Use a Gaussian Mixture Model to analyze and cluster the click data based on the number of clicks from different locations, aiming to identify distinct patterns of user behavior across 10 countries.

**Dataset**:
- **Click Data**: DataFrame with columns `link_id`, `location`, and `number_of_clicks`.

**Steps**:

1. **Preprocess Data**:
   - Ensure the dataset contains 10 distinct countries.
   - Create a matrix where rows represent different links and columns represent the number of clicks from each country.
   - Normalize the number of clicks to account for different scales.

2. **Fit GMM**:
   - Apply a Gaussian Mixture Model (GMM) to the click data matrix to identify clusters of links with similar click patterns across different countries.

3. **Evaluate Model**:
   - Analyze the resulting model by calculating the log lokelihood, BIC and AIC metrics.


In [157]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

file_path = '/content/drive/MyDrive/gmm-dataset.csv'
df = pd.read_csv(file_path)

urls = df.iloc[:, 0].values
click_data = df.iloc[:, 1:].values

# Normalize the click data to account for different scales
scaler = StandardScaler()
normalized_clicks = scaler.fit_transform(click_data)

# Normalize the number of clicks to account for different scales
scaler = StandardScaler()
normalized_clicks = scaler.fit_transform(click_data)


num_clusters = 11
gmm = GaussianMixture(n_components=num_clusters, random_state=0)
gmm.fit(normalized_clicks)

log_likelihood = gmm.score(normalized_clicks)
bic = gmm.bic(normalized_clicks)
aic = gmm.aic(normalized_clicks)

print(f"Log-Likelihood: {log_likelihood}")
print(f"BIC: {bic}")
print(f"AIC: {aic}")

Log-Likelihood: 0.06996612318933304
BIC: 3324.7551602035
AIC: 1436.0067753621333


In [158]:
from sklearn.metrics import silhouette_score
labels = gmm.predict(normalized_clicks)
silhouette_avg = silhouette_score(normalized_clicks, labels)
print(f"Silhouette Score: {silhouette_avg}")

Silhouette Score: 0.11044857831241178
